<a href="https://colab.research.google.com/github/rahul0772/python-ml-ai-relearning/blob/main/AI%20and%20ML%20with%20PyTorch/day15_Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# ## 1. Load the MNIST Dataset

# Define a transformation to normalize the images to a range of [0, 1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST training and test datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for training and test sets
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# ## 2. Define the CNN Model

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional Layer 1: 1 input channel (grayscale), 32 output channels (feature maps), kernel size of 3x3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Pooling Layer 1: Max pooling with 2x2 window
        self.pool = nn.MaxPool2d(2, 2)
        # Convolutional Layer 2: 32 input channels, 64 output channels
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Fully connected layer: Flatten the output from the convolutional layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Flattened dimension
        # Output layer: 10 classes (digits 0-9)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Apply convolutional layer 1 + activation function + pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # Apply convolutional layer 2 + activation function + pooling
        x = self.pool(torch.relu(self.conv2(x)))
        # Flatten the output to feed into fully connected layers
        x = x.view(-1, 64 * 7 * 7)
        # Apply fully connected layer 1 + activation function
        x = torch.relu(self.fc1(x))
        # Apply output layer (no activation here as we will use Cross-Entropy Loss)
        x = self.fc2(x)
        return x

# Instantiate the CNN model
model = CNN()

# ## 3. Set Up the Optimizer and Loss Function

# Using Adam optimizer and CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ## 4. Training the Model

# Number of epochs
epochs = 5

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()   # Zero the gradients
        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()         # Backpropagation
        optimizer.step()        # Update model parameters

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print the statistics for this epoch
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# ## 5. Evaluate on the Test Set

# Set the model to evaluation mode
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print the accuracy on the test set
print(f"\nTest Accuracy: {100 * correct / total:.2f}%")

# ## 6. Visualize Some Test Images with Predictions

# Get a batch of test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# Get the model's predictions for the batch
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# Plot some images along with their true labels and predicted labels
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i in range(5):
    ax = axes[i]
    ax.imshow(images[i].squeeze(), cmap="gray")
    ax.set_title(f"True: {labels[i].item()}, Pred: {predicted[i].item()}")
    ax.axis('off')
plt.show()

100%|██████████| 9.91M/9.91M [00:00<00:00, 49.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.72MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 15.0MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.66MB/s]


Epoch [1/5], Loss: 0.1620, Accuracy: 95.05%
Epoch [2/5], Loss: 0.0461, Accuracy: 98.59%
Epoch [3/5], Loss: 0.0321, Accuracy: 98.99%
Epoch [4/5], Loss: 0.0233, Accuracy: 99.27%
Epoch [5/5], Loss: 0.0191, Accuracy: 99.39%

Test Accuracy: 98.52%


AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'